# National Health and Nutrition Examination Survey (NHANES)
[https://wwwn.cdc.gov/nchs/nhanes/](https://wwwn.cdc.gov/nchs/nhanes/)

Este laboratório usará dados extraídos do NHANES.

## NHANES datasets from 2013-2014

Foram extraídos dados do [NHANES 2013-2014](https://wwwn.cdc.gov/nchs/nhanes/continuousnhanes/default.aspx?BeginYear=2013).

Os dados estão disponíveis em arquivos CSV no diretório `/data/nhanes`. Lá está disponível a documentação do esquema dos arquivos.

# Tarefa 1 - Esquema Relacional

Analise os arquivos CSV que estão no diretório `/data/nhanes` e escreva um esquema relacional referente às tabelas, suas chaves primárias e estrangeiras (não é SQL ainda). Coloque a chave primária com dois underlines antes e dois depois:

\__teste\__

demographic-person(\__id\__, gender, age),

medication-drug(\__code\__, name),

medications-use(person_id, drug_code, days_use) where person_id is foreign key from demographic-person and _drug_code_ is foreign key from medication-drug.

## Criação das Tabelas em SQL e Importação de Dados

Considere o seguinte exemplo em que é criada uma tabela em SQL e importado dados do arquivo CSV `demographic-person.csv`.

In [6]:
%defaultDatasource jdbc:h2:mem:db

In [7]:
CREATE TABLE Pessoa (
  Id VARCHAR(5) NOT NULL,
  Gender INTEGER,
  Age INTEGER,
  PRIMARY KEY(Id)
) AS SELECT
    id,
    gender,
    age
FROM CSVREAD('../data/demographic-person.csv');

In [ ]:
SELECT * FROM Pessoa;

# Tarefa 2 - Criação das Tabelas em SQL e Importação de Dados

Escreva instruções SQL para montar as duas tabelas restante, conforme definido no esquema relacional da **Tarefa 1**.

**Importante:** não defina chave primária para a tabela de uso de medicamento.

Durante a criação das tabelas, realize a importação dos dados do arquivo CSV nas tabelas criadas, conforme exemplo anterior.

In [9]:
CREATE TABLE Medicamento (
  Code VARCHAR(6) NOT NULL,
  Name VARCHAR(60),
  PRIMARY KEY(Code)
) AS SELECT
    code,
    name
FROM CSVREAD('../data/medications-drug.csv');

CREATE TABLE Uso_Medicamento (
  person_id VARCHAR(5) NOT NULL,
  drug_code VARCHAR(6) NOT NULL,
  days_use INTEGER,
  FOREIGN KEY(person_id)
    REFERENCES Pessoa(Id)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION,
  FOREIGN KEY(drug_code)
    REFERENCES Medicamento(Code)
      ON DELETE NO ACTION
      ON UPDATE NO ACTION
) AS SELECT
    person_id,
    drug_code,
    days_use
FROM CSVREAD('../data/medications-use.csv');

SELECT * FROM Medicamento;
SELECT * FROM Uso_Medicamento;

# Tarefa 3 - Consultas Simples

Execute as consultas a seguir.

a) Liste pessoas com idade acima de 60

In [10]:
SELECT Id, Age FROM Pessoa P WHERE P.Age > 60

b) Liste as idades das pessoas cadastradas (sem repetições)

In [11]:
SELECT Age, COUNT(*) Numero_pessoas FROM Pessoa P GROUP BY P.Age

c) Liste código das pessoas, idade, código dos medicamentos que usaram e dias de uso

In [12]:
SELECT UM.person_id, P.Age, UM.drug_code, UM.days_use FROM Uso_Medicamento UM, Pessoa P WHERE UM.person_id = P.id

d) Liste código das Pessoas, idade, nome dos medicamentos que usaram e quantos dias de uso

In [13]:
SELECT UM.person_id, P.Age, M.Name, UM.days_use FROM Uso_Medicamento UM, Pessoa P, Medicamento M WHERE UM.person_id = P.id AND UM.drug_code = M.code

e) Liste nome dos medicamentos que foram usados por pessoas (sem repetições)

In [14]:
SELECT M.Name FROM Medicamento M, Uso_Medicamento UM WHERE M.Code = UM.drug_code GROUP BY M.Name

f) Liste nome dos medicamentos usados por mais de 2.000 dias (sem repetições)

In [15]:
SELECT M.Name FROM Medicamento M, Uso_Medicamento UM WHERE M.Code = UM.drug_code AND UM.days_use > 2000 GROUP BY M.Name